In [8]:
import numpy as np
import tifffile as tiff

In [2]:
file = r"C:\Users\dongq\OneDrive\OCULAR Files\Image\05052023 mouse 3w\Image 000004.tif"

In [36]:
im = tiff.imread(file)

In [13]:
immetadata = tiff.TiffFile(file).pages[0].tags

In [15]:
for i in immetadata:
    print(i)

TiffTag 254 NewSubfileType @10 LONG @18 = UNDEFINED
TiffTag 256 ImageWidth @22 LONG @30 = 1376
TiffTag 257 ImageLength @34 LONG @42 = 1024
TiffTag 258 BitsPerSample @46 SHORT @54 = 16
TiffTag 259 Compression @58 SHORT @66 = NONE
TiffTag 262 PhotometricInterpretation @70 SHORT @78 = MINISBLACK
TiffTag 266 FillOrder @82 SHORT @90 = MSB2LSB
TiffTag 273 StripOffsets @94 LONG @102 = (258,)
TiffTag 277 SamplesPerPixel @106 SHORT @114 = 1
TiffTag 278 RowsPerStrip @118 LONG @126 = 1024
TiffTag 279 StripByteCounts @130 LONG @138 = (2818048,)
TiffTag 282 XResolution @142 RATIONAL @242 = (1376, 4)
TiffTag 283 YResolution @154 RATIONAL @250 = (1376, 4)
TiffTag 296 ResolutionUnit @166 SHORT @174 = INCH
TiffTag 339 SampleFormat @178 SHORT @186 = UINT
TiffTag 33821 @190 SHORT @198 = 60
TiffTag 33826 @202 ASCII[12041] @2818306 = <?xml version="1.0" encoding="UTF-8"
TiffTag 33827 @214 LONG @222 = 3
TiffTag 33828 @226 LONG @234 = 1


In [16]:
file2 = r"C:\Users\dongq\OneDrive\OCULAR Files\Image\0127\0128\40x-1.tif"

In [17]:
im2 = tiff.TiffFile(file2).pages[0].tags

In [18]:
for i in im2:
    print(i)

TiffTag 254 NewSubfileType @10 LONG @18 = UNDEFINED
TiffTag 256 ImageWidth @22 LONG @30 = 1376
TiffTag 257 ImageLength @34 LONG @42 = 1024
TiffTag 258 BitsPerSample @46 SHORT @54 = 16
TiffTag 259 Compression @58 SHORT @66 = NONE
TiffTag 262 PhotometricInterpretation @70 SHORT @78 = MINISBLACK
TiffTag 270 ImageDescription @82 ASCII[47] @206 = ImageJ=1.53t unit=\u00B5m min
TiffTag 273 StripOffsets @94 LONG @102 = (753,)
TiffTag 277 SamplesPerPixel @106 SHORT @114 = 1
TiffTag 278 RowsPerStrip @118 SHORT @126 = 1024
TiffTag 279 StripByteCounts @130 LONG @138 = (2818048,)
TiffTag 282 XResolution @142 RATIONAL @253 = (6000000, 1000000)
TiffTag 283 YResolution @154 RATIONAL @261 = (6000000, 1000000)
TiffTag 296 ResolutionUnit @166 SHORT @174 = NONE
TiffTag 50838 IJMetadataByteCounts @178 LONG[4] @269 = (20, 128, 136, 184)
TiffTag 50839 IJMetadata @190 BYTE[468] @285 = {'Overlays': [b'Iout\x00\xe4\x01


In [19]:
file3 = r"D:\Download\MAX_C2-604 (1).tif"
im3 = tiff.TiffFile(file3).pages[0].tags
for i in im3:
    print(i)


TiffTag 254 NewSubfileType @10 LONG @18 = UNDEFINED
TiffTag 256 ImageWidth @22 LONG @30 = 512
TiffTag 257 ImageLength @34 LONG @42 = 512
TiffTag 258 BitsPerSample @46 SHORT @54 = 16
TiffTag 262 PhotometricInterpretation @58 SHORT @66 = MINISBLACK
TiffTag 270 ImageDescription @70 ASCII[47] @182 = ImageJ=1.51n unit=micron min=
TiffTag 273 StripOffsets @82 LONG @90 = (1781,)
TiffTag 277 SamplesPerPixel @94 SHORT @102 = 1
TiffTag 278 RowsPerStrip @106 SHORT @114 = 512
TiffTag 279 StripByteCounts @118 LONG @126 = (524288,)
TiffTag 282 XResolution @130 RATIONAL @229 = (795647, 1000000)
TiffTag 283 YResolution @142 RATIONAL @237 = (795647, 1000000)
TiffTag 296 ResolutionUnit @154 SHORT @162 = NONE
TiffTag 320 ColorMap @166 SHORT[768] @245 = array([[0, 0, 0, ..., 0, 0, 0], [0,


In [20]:
immetadata['XResolution'].value

(1376, 4)

In [21]:
resolution = (6000000, 1000000)
unit = 'micron'

In [37]:
scale = np.max(im) / 255 + 1
im = im/scale
im = im.astype(np.uint8)

In [38]:
im.max()

252

In [40]:
# Create a new dictionary to store modified metadata
new_metadata = {}

# Copy metadata tags to the new dictionary
for tag in immetadata:
    new_metadata[tag.name] = tag.value
new_metadata['XResolution'] = resolution
new_metadata['YResolution'] = resolution
new_metadata['ResolutionUnit'] = unit

In [42]:
tiff.imwrite('new_image.tif', im, metadata=new_metadata)
